In [32]:
import sys
import pandas as pd
import os
from datetime import datetime
import numpy as np
import json

# 파일전송용 Library -----------
import time       #시간지연 library사용
import paramiko   #sftp파일전송 library사용
# ------------------------------

# Bayesian Optimization --------
from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours

BO_RangeFile='/home/tangosvc/code/data/BO_Range2.csv'
Opt_statusFile='opt_status2.json'

requestPrefix = "SON_TO_TEOS_RL_REQUEST_"
resultPrefix = "TEOS_TO_SON_RL_RESULT_" 

cellcode=['2019Y9667','2019Z2137','2019Z3737', '20193BHZH', \
         '20193BHTY', '20193AMR7', '2019Y9678', '2019Z3222', \
         '20193BIEF', '20193BI6Z', '2019Z2954', '2019Z2586', \
         '2019Z0388', '2019Z2764', '2019Z2779', '2019Y9862',]
# ------------------------------

In [33]:
class TEOS_filegetput:
        def __init__(self, ML_dirnm, TEOS_indirnm, TEOS_outdirnm, filetime):
            # ML서버내 파일관리폴더(Request, Result모두 동일폴더에서 관리함)
            self.ML_dirnm=ML_dirnm
            # TEOS관리폴더(Request, Result 별도로 관리함)
            self.TEOS_indirnm=TEOS_indirnm
            self.TEOS_outdirnm=TEOS_outdirnm
            # 시간기준으로 폴더생성/파일관리
            self.time=filetime        
            
        # dirnm에 있는 filenm으로 시작하는 모든 파일목록 생성
        def get_filelist(self, dirnm, filenm):
            file_list=[f for f in os.listdir(dirnm) if f.startswith(filenm)]
            return(file_list)
        
        # 파일존재시 파일명리턴 부재시 공백리턴
        def get_fileexist(self, dirnm, filenm):
            if os.path.exists(dirnm+'/'+filenm):
                return(filenm)
            else:
                return('')    
        
        # TEOS서버접속
        def conn_TEOS_svr(self):
            host='90.90.220.8'
            port=10003
            transport=paramiko.Transport((host,port))
            password='@Son1234!1'
            username='son'
            transport.connect(username=username, password=password)
            sftp=paramiko.SFTPClient.from_transport(transport)
            return(sftp, transport)

        # put명령실행            
        def do_requestfile_put(self, sftp, transport, putfilenm):
            fromfile=self.ML_dirnm+'/'+self.time+'/'+putfilenm
            tofile=self.TEOS_indirnm+'/'+self.time+'/'+putfilenm
            
            # TEOS서버에 복사할 폴더가 존재하지 않으면 생성함
            dirlist=sftp.listdir(self.TEOS_indirnm)             
            if not self.time in dirlist:
                sftp.mkdir(self.TEOS_indirnm+'/'+self.time)

            # ML서버에 복사할 파일이 존재하지 않으면 실행종료
            temp=os.path.split(fromfile)
            if self.get_fileexist(temp[0], temp[1])!='':
                sftp.put(fromfile, tofile)
                return(1)
            else:
                print('ML서버에 Put대상 파일(%s)이 없어 TEOS서버에 Put명령을 하지 못했습니다.\n' %fromfile)                
                return(0)

        # get명령실행    
        def do_resultfile_get(self, sftp, transport, getfilenm):         
            fromfile=self.TEOS_outdirnm+'/'+self.time+'/'+getfilenm
            tofile=self.ML_dirnm+'/'+self.time+'/'+getfilenm            
            done_normal=0
            
            # ML서버에 복사해올 폴더가 존재하지 않으면 생성함
            if not os.path.exists(self.ML_dirnm+'/'+self.time):
                os.mkdir(self.ML_dirnm+'/'+self.time)
            
            # TEOS내에 출력파일 저장폴더 미생성시 60초간격으로 20번 확인
            for i in range(60):
                if self.time in sftp.listdir(self.TEOS_outdirnm):
                    for j in range(60):
                        # TEOS내에 출력파일 존재하지 않으면 60초간격으로 20번 확인
                        dirlist=sftp.listdir(self.TEOS_outdirnm+'/'+self.time)                        
                        if getfilenm in dirlist:      
                            sftp.get(fromfile, tofile)
                            done_normal=1
                            break
                        else:
                            print('파일 %s이 없어 60초 대기후 다시 확인합니다(최대60회).' %getfilenm)
                            time.sleep(60)
                    break
                else:
                    print('폴더 %s이 없어 60초 대기후 다시 확인합니다(최대60회).' %self.time)
                    time.sleep(60)
            return(done_normal)

        # sftp접속해제
        def close_TEOS_svr(self, sftp, transport):
            sftp.close()
            transport.close()
            
        def RequestFile_Put(self, filenm):
            try:
                sftp, transport=self.conn_TEOS_svr()      
                done_normal=self.do_requestfile_put(sftp, transport, filenm)
                self.close_TEOS_svr(sftp, transport)
                return(done_normal)

            except:
                print('Error : TEOS server connection')
                self.close_TEOS_svr(sftp, transport)  
                return(0)

        def ResultFile_Get(self, filenm): 
            try:
                sftp, transport=self.conn_TEOS_svr()
                done_normal=self.do_resultfile_get(sftp, transport, filenm)
                self.close_TEOS_svr(sftp, transport)
                return(done_normal)

            except:
                print('Error : TEOS server connection')
                self.close_TEOS_svr(sftp, transport)
                return(0)

In [34]:
# ML_dirnm='/home/tangosvc/code/data'
class Request_filecreate:
    def __init__(self, ML_dirnm, time, seq):
        # ML서버내 파일관리폴더(Request, Result모두 동일폴더에서 관리함)
        self.ML_dirnm=ML_dirnm
        # 시간기준으로 폴더생성/파일관리
        self.time=time
        self.seq=str(seq)
        
        self.pre_reqfilenm=''
        self.pre_resultfilenm=''         
            
    # Site정보 불러오기(site_info.csv파일내용 변경으로 수정함 -19/10/31)
    def getSiteInfo(dofilter=0) :
        filenm =ML_dirnm+'/'+'site_info.csv'
        try:
            df = pd.read_csv(filenm, encoding = 'utf-8', header = 1) 
            df = df[['Sisul code', 'target']]

            if dofilter==1:
                crit=(df['target']=='Y')
                df = df[crit]

            df.columns = ['SISUL_CD1', 'target']
            df = df.dropna(how ='all')
        except FileNotFoundError:
            pass
        return(df)

    # 결과파일 불러온 후  근접셀 찾아 리턴하기(NBR Site계산로직변경으로 신규추가함 - 19/10/31)
    def getNBRSiteInfo():
        filenm = self.pre_resultfilenm
        try:
            df = pd.read_csv(filenm, encoding = 'utf-8', sep='|', header=None) 
            df.columns = ['REQUEST_DATE','SCENARIO_SEQ','REQUEST_SEQ','XPOS','YPOS',
                                      'SISUL_CD1','RSRP1','SINR1',
                                      'SISUL_CD2','RSRP2','SINR2',
                                      'SISUL_CD3','RSRP3','SINR3']

            input1=self.getSiteInfo()
            f={lambda x: pd.Series.mode(x)[0]}
            input2=df.groupby(['SISUL_CD1'])['SISUL_CD2'].agg(f)
            input2.columns=['NBR_SISUL_CD']
            df = pd.merge(input1, input2, how = 'left', left_on = 'SISUL_CD1', right_on= 'SISUL_CD1')

        except FileNotFoundError:
            pass
        return(df)    


    # Read Result File 
    def getResult(self) :
        resultFile=''
        resultPrefix = "TEOS_TO_SON_RL_RESULT_"
        filenm=self.pre_resultfilenm

        try:
            resultFile=pd.read_csv(filenm, sep='|', encoding = 'utf-8', header=None)
            resultFile.columns = ['REQUEST_DATE','SCENARIO_SEQ','REQUEST_SEQ','XPOS','YPOS',
                          'SISUL_CD1','RSRP1','SINR1',
                          'SISUL_CD2','RSRP2','SINR2',
                          'SISUL_CD3','RSRP3','SINR3']
        except FileNotFoundError:
            print("Request파일(%s)이 존재하지 않아 실행을 중단합니다.\n" %filenm)
            pass
        return(resultFile)
    
    
# State Quality 계산(동적근접셀 선정기능 포함, 근접셀 정보를 Result파일에서 Dynamic하게 추출함)-----------------------------------------------------
    def getStateQuality(self, wa = 0.5, wb = 0.5) :
        sqResult=''
        result = self.getResult()

        # Cell SINR avg , 5%
        def q5(x):
            return x.quantile(0.05)

        # 대상셀에 대한 SQ를 계산하여 SISUL_CD1을 ID로 하는 sqResult DF에 저장하기
        f = {'SINR1': ['mean', q5], 'RSRP1': ['mean']}
        sqResult = result.groupby('SISUL_CD1').agg(f)
        sqResult['sq'] = sqResult[('SINR1', 'mean')] * wa + (1-wa) * sqResult[('SINR1', 'q5')]
        sqResult=sqResult.reset_index()
    
# rs : ['SINR1_Mean', 'SINR1_Q5', 'RSRP1_Mean', 'sq', 'nbrSq', 'gsq']

        # 결과파일에 근거하여 근접셀 계산후 GSQ계산
        # 결과파일에서 SISUL_CD1, sq 추출
        input1=sqResult[['SISUL_CD1','sq']]
        
        # SISUL_CD1별 근접셀 계산
        f={lambda x: pd.Series.mode(x)[0]}
        input2=result.groupby(['SISUL_CD1'])['SISUL_CD2'].agg(f)
        input2.columns=['NBR_SISUL_CD']
        input2=input2.reset_index()
        df = pd.merge(input1, input2, how = 'left', left_on = 'SISUL_CD1', right_on= 'SISUL_CD1')

        # input1의 열명변경 : sq --> nbrSQ
        # df열구성 : SISUL_CD1, sq, NBR_SISUL_CD
        input1.columns=['SISUL_CD1', 'nbrSq']
        df = pd.merge(df, input1, how='left', left_on='NBR_SISUL_CD', right_on='SISUL_CD1')
        # df열구성 : SISUL_CD1, sq, NBR_SISUL_CD, nbrSq
        sqResult['nbrSq']=df['nbrSq']

        # Global SQ
        # sqResult['gsq'] = sqResult['sq'] * wb + (1 - wb) * sqResult[('nbrSq')]
        # 근접셀이 없는 경우 대상셀의 GSQ수식을 적용하지 않고 sq값을 리턴함
        # 근접셀이 있는 경우 GSQ수식을 적용한 결과값을 리턴함
        sqResult['gsq'] = np.where(sqResult['nbrSq'] == 99999, sqResult['sq'] , sqResult['sq'] * wb + (1 - wb) * sqResult[('nbrSq')])
        sqResult.columns =  ['SISUL_CD', 'SINR1_Mean', 'SINR1_Q5', 'RSRP1_Mean', 'sq', 'nbrSq', 'gsq'] 
        
        # rs : ['SISUL_CD  SINR1_Mean  SINR1_Q5  RSRP1_Mean sq  nbrSq   gsq]
        SRC_cell=pd.DataFrame(cellcode, columns=['cellcode']) 
#         print(sqResult, SRC_cell)        
        
        gsq_each=pd.merge(SRC_cell, sqResult, how='left', left_on='cellcode', right_on='SISUL_CD')['gsq']
        gsq_avg=gsq_each.mean()
        return(sqResult, gsq_avg, gsq_each)
    # rs : ['SISUL_CD  SINR1_Mean  SINR1_Q5  RSRP1_Mean sq  nbrSq   gsq]
    # --------------------------------------------------------------------------------------------------------

    # Read Request File
    def getRequest(self) :
        rq=''
        requestPrefix = "SON_TO_TEOS_RL_REQUEST_"
        filenm=self.pre_reqfilenm
        try:
            rq = pd.read_csv(filenm, sep = '|', encoding = 'utf-8', header = None)            
            rq.columns = ['REQUEST_DATE', 'SCENARIO_SEQ', 'REQUEST_SEQ', 
                          'START_XPOS', 'END_XPOS', 'START_YPOS', 'END_YPOS', 'SITE_NAME', 'FA_SEQ', 
                          'SISUL_CD', 'DUH_EQUIP_ID', 'PCI', 'XPOS', 'YPOS', 'CELL_NO', 'TOWER_HEIGHT', 
                          'BUILDING_HEIGHT', 'ANTENNA_NM', 'ORIENT', 
                          'E_TILT', 'MTILT']
        except FileNotFoundError:
            print("Request파일(%s)이 존재하지 않아 실행을 중단합니다.\n" %filenm)
            pass

        return(rq)

    # get Current State from Request FIle 
    def getState(self):
        state = self.getRequest()
        
        # Src E-Tilt
        etilt = state['E_TILT']
        bins = [-8, -6, -4, -2, 0, 1, 3, 5, 7, 9]
#        bins = [-5, -4, -2, 0, 1, 3, 5, 6]
        labels = ["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9"]
        cats = pd.cut(etilt, bins, labels=labels, include_lowest=True, right=False)
        state['SRC_STATE'] = cats

        # Nbr셀찾아 TILT와 STATE설정
        def getNbrState(x):
            # SISUL Code정보
            nbrSite = self.getSiteInfo()
            tmp = list(state.SISUL_CD)
            nbrCell = list(nbrSite.NBR_CD1[nbrSite.SISUL_CD1 == x])

            if len(nbrCell) == 1:
                nbrState = state.E_TILT[tmp.index(nbrCell[0])]
            else:
                nbrState = 99999
            return nbrState
        
        # Nbr E-Tilt
        state['NBR_TILT'] = list(map(lambda i: getNbrState(i), state.SISUL_CD.to_list()))
        catsNbr = pd.cut(state['NBR_TILT'], bins, labels=labels, include_lowest=True, right=False)
        state['NBR_STATE'] = catsNbr

        return(state)

    def cutStateQuality(self, df2): 
        sinr = df2['SINR1_Mean']
        bins = [-999, 0, 5, 10, 15, 20, 25, 99]
    #     labels = ["s1", "s2", "s3", "s4", "s5", "s6", "s7"]
    #     cats = pd.cut(sinr, bins, labels=labels, include_lowest=True, right=True)
        cats = pd.cut(sinr, bins, include_lowest=True, right=True)
        df2['SINR1_STATE'] = cats

        #rsrp
        rsrp = df2['RSRP1_Mean']
        bins = [-999, -110, -100, -90, -80, -70, -60, 999]
    #    labels = ["s1", "s2", "s3", "s4", "s5", "s6", "s7"]
    #     cats = pd.cut(sinr, bins, labels=labels, include_lowest=True, right=False)
        cats = pd.cut(rsrp, bins, include_lowest=True, right=True)
        df2['RSRP1_STATE'] = cats
        df2 = df2[['SISUL_CD', 'SINR1_STATE', 'RSRP1_STATE']]    
        return(df2)

    
#     def Sim_Wait_Time(self, t1, s_time, e_time):
# #         Sim_Time_From=datetime.strptime(s_time, '%H:%M')
# #         Sim_Time_To=datetime.strptime(e_time, '%H:%M')
# # Cur_Time, Sim_Time_From, Sim_Time_To

#         if(t1>s_time) & (t1<e_time):
#             return(0)
#         else:
#             timedelta=datetime.strptime('23:59','%H:%M')-datetime.strptime(t1,'%H:%M')
#             return(int(timedelta.total_seconds())+360)
        
    
    # update and read request File(Parameter tilt는 데이터 프레임임)
    def outputDatFile(self, action = None, tilt=None ): 
        output = self.getRequest()
        
        # 구현필요 : ------------ 모델을 통하여 action을 받아와야함(BO or RL모델링 수행함)
        # RL은 현 tilt에 대한 변경값
        # BO는 tilt값
        if tilt.empty:
            if action is None :
                action = np.random.randint(-2, 2, output.shape[0])
                action = pd.DataFrame(action)[0]

            # action은 현 tilt값에 대한 변화량
            # action적용결과 E-Tilt 가 -8~8 범위를 넘어가는 경우 범위내로 강제 action조정-----       
            action = np.where((action + output['E_TILT'])<-8, -8-output['E_TILT'], action)
            action = np.where((action + output['E_TILT'])>8, 8-output['E_TILT'], action)
            output['E_TILT'] = action + output['E_TILT']
        else:
            x11=output['SISUL_CD'].tolist()
            y11=tilt['SISUL_CD1'].tolist()
            y12=tilt['E_TILT'].tolist()
            
            for i in range(len(y11)):
                selrow=x11.index(y11[i])
                output['E_TILT'][selrow]=y12[i]

#             print(output['SISUL_CD'])
#             print(output['E_TILT'])
#             print(tilt['E_TILT'])
#             print(tilt['SISUL_CD1'])                

        output['REQUEST_SEQ'] = self.seq
        output['REQUEST_DATE']= self.time
        #-------------------------------------------------        
        
        if not os.path.isdir(self.ML_dirnm+'/'+self.time):
            os.mkdir(self.ML_dirnm+'/'+self.time)

        filenm = self.ML_dirnm+'/'+self.time+'/'+requestPrefix + self.time + "_[" + self.seq + "].dat"
        print("Request 파일(%s)을 생성합니다.\n" %filenm)
        output.to_csv(filenm, sep = "|", header = False, index = False)      

In [35]:
# TEOS 연동모듈(Request File + Result File 주고받기))
def Do_simul_NEXT(filetime, a): 
    # 파일저장위치(ML서버 : request/result파일 한개 폴더에서 관리, TEOS서버 : request/result파일 별도 폴더에서 관리)
    ML_dirnm='/home/tangosvc/code/data'
    TEOS_indirnm='/disk2/interworking_data/SON/EngAnalysis/request'        
    TEOS_outdirnm='/disk2/interworking_data/SON/EngAnalysis/result'       

    #sftp라이브러리 paramiko의 실행 log저장/관리
    paramiko.util.log_to_file('/home/tangosvc/code/data/paramiko.log')
    
    try:    
        # SEQ+1차수의 시뮬레이션 실행(실행가능 시간인지 확인후 실행함)
        b=TEOS_filegetput(ML_dirnm, TEOS_indirnm, TEOS_outdirnm, filetime)        
        
        # request file에 대한 put명령 실행
        filenm = requestPrefix + filetime + "_[" + a.seq + "].dat"     
        Putresult=b.RequestFile_Put(filenm)

         # Request Get
         # Put명령이 성공했을 경우 -> result file에 대한 get명령 실행
        if Putresult==1:            
            filenm = resultPrefix + filetime + "_[" + a.seq + "].dat"
            print('%s에 대한 get명령을 실행합니다.' %filenm)  
            b.ResultFile_Get(filenm)

            a.pre_reqfilenm=ML_dirnm+'/'+filetime+'/'+filenm              
            a.pre_resultfilenm=ML_dirnm+'/'+filetime+'/'+filenm  
            del b  
            return(1)
        else:
            del b  
            return(0)      
    except:
        return(0)

In [36]:
# BO용 모듈
para_no=16
    
# Pbound설정
def get_pbounds():
    df1=pd.read_csv(BO_RangeFile, sep=',', index_col='varnm')
    df1=df1.to_dict('split')
    pbounds=dict(zip(df1['index'], df1['data']))
    return(pbounds)

# 대구지역 cell 16개의 tilt값
def black_box_function(c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16):   
    # Parameter 입력값
    c1=round(c1,0);c2=round(c2,0);c3=round(c3,0);c4=round(c4,0);c5=round(c5,0)
    c6=round(c6,0);c7=round(c7,0);c8=round(c8,0);c9=round(c9,0);c10=round(c10,0)
    c11=round(c11,0);c12=round(c12,0);c13=round(c13,0);c14=round(c14,0);c15=round(c15,0)
    c16=round(c16,0)
    input_val=[c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16]
    req_tilt=pd.DataFrame([input_val, cellcode]).T
    req_tilt.columns=(['E_TILT', 'SISUL_CD1'])    

    
    

    
    # 시뮬레이션 운영시간
    Sim_Time_From='00:20'
    Sim_Time_To='23:20' 

    #-------------------------------------------------------------
    # Simulation중단시간 Check -> 대기(00:05~23:25이외의 시간) 
    Cur_Time=datetime.now().strftime('%H:%M') 
    while (Cur_Time < Sim_Time_From) | (Cur_Time > Sim_Time_To):
#         wait_time=a.Sim_Wait_Time(Cur_Time, Sim_Time_From, Sim_Time_To)
        time.sleep(180)
        Cur_Time=datetime.now().strftime('%H:%M') 
    #-------------------------------------------------------------

    
    
    
    # Parameter 입력값에 따른 출력값
    y=-9999
    while y==-9999:
        # 시뮬레이션 실행코드
        y=cal_obj_fun(req_tilt)*100
    return(y)

In [37]:
def set_optinfo(ML_dirnm, seq, pre_reqfilenm, pre_resultfilenm):
    destination=ML_dirnm+'/'+Opt_statusFile
#     data=pd.DataFrame([seq], index=['SEQ'], columns=['value'])
#     data.to_json(destination)

    with open(destination, encoding='UTF8') as data_file:
        data=json.load(data_file)
    data['value']['SEQ']=seq
    data['value']['pre_reqfilenm']=pre_reqfilenm
    data['value']['pre_resultfilenm']=pre_resultfilenm
    with open(destination, 'w') as txtfile:
        json.dump(data, txtfile)
        
def get_optinfo(ML_dirnm):
    destination=ML_dirnm+'/'+Opt_statusFile
    with open(destination, encoding='UTF8') as data_file:
        data=json.load(data_file)
    return(data['value'])

In [38]:
# BO용 모듈
def cal_obj_fun(req_tilt):    
    #값 초기화---------------------------------
    # request, result file 관리 폴더(ML서버, TEOS서버)
    ML_dirnm='/home/tangosvc/code/data'    
    # simulation 실행일(파일생성일)
    filetime=datetime.now().strftime('%Y%m%d') 
    #----------------------------------------------  
    
    # TEOS 최근 적용 파일명, Seq 정보를 json파일에서 가져오기
    seq=get_optinfo(ML_dirnm)['SEQ']
    pre_reqfilenm=get_optinfo(ML_dirnm)['pre_reqfilenm']
    pre_resultfilenm=get_optinfo(ML_dirnm)['pre_resultfilenm']
    
#     print(seq, pre_reqfilenm, pre_resultfilenm)
#     seq=47
#     # 시뮬레이션 시작용 Request, Result 파일
#     pre_reqfilenm='/home/tangosvc/code/data/20191031/SON_TO_TEOS_RL_REQUEST_20191031_[%s].dat' %str(seq)
#     pre_resultfilenm='/home/tangosvc/code/data/20191031/TEOS_TO_SON_RL_RESULT_20191031_[%s].dat' %str(seq)
    # ------------------------------------------------------------    
    
    # Request_filecreate 클래스를 생성함
    a=Request_filecreate(ML_dirnm, filetime, seq)
    a.pre_reqfilenm=pre_reqfilenm
    a.pre_resultfilenm=pre_resultfilenm    
    a.seq=str(seq+1)    
    a.outputDatFile(action = None, tilt=req_tilt) 
    
    Do_simul_NEXT(filetime, a)
    # 시점 t : state(SRC tilt, NBR tilt, SINR, RSRP) 계산 ------------------------------
    # state(tilt1의 SINR, RSRP)정보 - ResultFile에서 수집(SISUL_CD기준으로 계산)
    rs, gsq_avg,_ = a.getStateQuality(wa = 0.5, wb = 0.5) 

    # rs : ['SISUL_CD  SINR1_Mean  SINR1_Q5  RSRP1_Mean sq  nbrSq   gsq]  
    set_optinfo(ML_dirnm, seq+1, \
                ML_dirnm+'/'+filetime+'/'+requestPrefix+ filetime+'_'+'['+str(seq+1)+'].dat', \
                ML_dirnm+'/'+filetime+'/'+resultPrefix+ filetime+'_'+'['+str(seq+1)+'].dat')
    return(gsq_avg)

In [ ]:
# ============================================================================================
# BO 실행
# Bayesian Optimization 수행
if __name__=="__main__":
    print(Colours.yellow("---Parameter Exploration Range(min, max) ---"))
# 탐색범위 설정(pbounds)
    pbounds=get_pbounds()
    print(pbounds)
    print(Colours.yellow("---Optimizing TEOS by BO ---"))
    optimizer=BayesianOptimization(
        # 목적함수(Surrogate Function) 설정
        f=black_box_function,
        # 탐색영역 설정
        pbounds=pbounds,
        # 재현성을 위한 Random Seed 설정
        random_state=1234,
        verbose=2 #옵션0 : 미출력, 1 : 일부출력, 2 : 전체이력 출력
    )
    
# 초기 탐색Iteration과 Iteration반복회수 설정
# 반복옵션이 반영됨
    optimizer.maximize(init_points=15, n_iter=200)
    op_result=optimizer.max

#Iteration 종료시 후처리
    print("Final result : ", op_result)
# BO결과값 처리    
    a2=op_result['params']
    a3=pd.DataFrame([a2['c'+str(i+1)] for i in range(para_no)], \
                index=['c'+str(i+1) for i in range(para_no)])
    a3=round(a3)
    print(a3)

---Parameter Exploration Range(min, max) ---
{'c1': [0, 8], 'c2': [0, 8], 'c3': [0, 8], 'c4': [0, 8], 'c5': [0, 8], 'c6': [0, 8], 'c7': [0, 8], 'c8': [0, 8], 'c9': [0, 8], 'c10': [0, 8], 'c11': [0, 8], 'c12': [0, 8], 'c13': [0, 8], 'c14': [0, 8], 'c15': [0, 8], 'c16': [0, 8]}
---Optimizing TEOS by BO ---
|   iter    |  target   |    c1     |    c10    |    c11    |    c12    |    c13    |    c14    |    c15    |    c16    |    c2     |    c3     |    c4     |    c5     |    c6     |    c7     |    c8     |    c9     |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


/home/tangosvc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Request 파일(/home/tangosvc/code/data/20191112/SON_TO_TEOS_RL_REQUEST_20191112_[1058].dat)을 생성합니다.

TEOS_TO_SON_RL_RESULT_20191112_[1058].dat에 대한 get명령을 실행합니다.
|  1        |  163.9    |  1.532    |  4.977    |  3.502    |  6.283    |  6.24     |  2.181    |  2.212    |  6.415    |  7.665    |  7.007    |  2.863    |  4.008    |  5.468    |  5.702    |  2.962    |  4.49     |
Request 파일(/home/tangosvc/code/data/20191112/SON_TO_TEOS_RL_REQUEST_20191112_[1059].dat)을 생성합니다.

TEOS_TO_SON_RL_RESULT_20191112_[1059].dat에 대한 get명령을 실행합니다.
파일 TEOS_TO_SON_RL_RESULT_20191112_[1059].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1059].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1059].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1059].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1059].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1059].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESU

파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1065].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
|  8        |  170.3    |  3.662    |  7.363    |  7.033    |  2.021    |  2.784    |  1.461    |  7.214    |  5.652    |  5.813    |  7.201    |  6.233    |  4.793    |  2.329    |  1.211    |  2.681    |  5.26     |
Request 파일(/home/tangosvc/code/data/201911

파일 TEOS_TO_SON_RL_RESULT_20191112_[1071].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
|  14       |  137.3    |  2.257    |  2.68     |  0.6682   |  6.087    |  4.074    |  5.288    |  5.043    |  2.967    |  3.574    |  3.321    |  3.843    |  7.867    |  2.987    |  0.09926  |  7.375    |  6.986    |
Request 파일(/home/tangosvc/code/data/20191112/SON_TO_TEOS_RL_REQUEST_20191112_[1072].dat)을 생성합니다.

TEOS_TO_SON_RL_RESULT_20191112_[1072].dat에 대한 get명령을 실행합니다.
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191112_[1072].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_2019111

...

Request 파일(/home/tangosvc/code/data/20191113/SON_TO_TEOS_RL_REQUEST_20191113_[1152].dat)을 생성합니다.

TEOS_TO_SON_RL_RESULT_20191113_[1152].dat에 대한 get명령을 실행합니다.
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1152].dat이 없어 60초 대기후 다시 확인합니다(최대60회).


/home/tangosvc/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


|  95       |  208.7    |  2.603    |  5.042    |  6.674    |  2.598    |  4.004    |  1.777    |  7.01     |  5.145    |  4.481    |  5.874    |  7.934    |  5.094    |  2.439    |  3.147    |  4.095    |  5.049    |


/home/tangosvc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Request 파일(/home/tangosvc/code/data/20191113/SON_TO_TEOS_RL_REQUEST_20191113_[1153].dat)을 생성합니다.

TEOS_TO_SON_RL_RESULT_20191113_[1153].dat에 대한 get명령을 실행합니다.
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).
파일 TEOS_TO_SON_RL_RESULT_20191113_[1153].dat이 없어 60초 대기후 다시 확인합니다(최대60회).


In [827]:
# ============================================================================================
# RL실행
if __name__ == "__main__":    
    Cellcode=['2019Y9667','2019Z2137','2019Z3737', '20193BHZH', \
             '20193BHTY', '20193AMR7', '2019Y9678', '2019Z3222', \
             '20193BIEF', '20193BI6Z', '2019Z2954', '2019Z2586', \
             '2019Z0388', '2019Z2764', '2019Z2779', '2019Y9862',] 
    
    # 분석시마다 사용자가 입력해줘야 하는 값 ---------------------
    # 시작 SEQ & 시간정보
    seq=47
    # 시뮬레이션 시작용 Request, Result 파일
    pre_reqfilenm='/home/tangosvc/code/data/20191031/SON_TO_TEOS_RL_REQUEST_20191031_[%s].dat' %str(seq)
    pre_resultfilenm='/home/tangosvc/code/data/20191031/TEOS_TO_SON_RL_RESULT_20191031_[%s].dat' %str(seq)
    # ------------------------------------------------------------    

    
    
    
    
    #값 초기화---------------------------------
    # request, result file 관리 폴더(ML서버, TEOS서버)
    ML_dirnm='/home/tangosvc/code/data'    
    
    # simulation 실행일(파일생성일)
    filetime=datetime.now().strftime('%Y%m%d') 
    #----------------------------------------------  

    # Request_filecreate 클래스를 생성함
    a=Request_filecreate(ML_dirnm, filetime, seq)
    a.pre_reqfilenm=pre_reqfilenm
    a.pre_resultfilenm=pre_resultfilenm
    # simulation 1회 iteration별로 수행 agent개수(동시적용 request file 개수))
    iteration_no=1
    # 초기 Request File에 대한 시뮬레이션은 Result 파일없이 진행함
    # 초기 Request File 시뮬레이션이 아닌 경우에는 firstrequest값을 1이 아닌값으로 설정함
    firstrequest=1
    
    while int(a.seq) < seq+iteration_no :
#         print(a.seq, seq+iteration_no)
        if not firstrequest==1:
            if not(os.path.exists(a.pre_reqfilenm)):
                print('%s파일이 존재하지 않아 실행을 중단합니다.' %a.pre_reqfilenm)
                break

            if not(os.path.exists(a.pre_resultfilenm)):
                print('%s파일이 존재하지 않아 실행을 중단합니다.' %a.pre_resultfilenm)
                break


            # 시점 t : state(SRC tilt, NBR tilt, SINR, RSRP) 계산 ------------------------------
            # state(tilt1, tilt2) 정보 - Request File에서 수집
            rq = a.getState()
            # state(tilt1의 SINR, RSRP)정보 - ResultFile에서 수집(SISUL_CD기준으로 계산)
            rs, gsq_avg,_ = a.getStateQuality(wa = 0.5, wb = 0.5) 
            # rs : ['SISUL_CD  SINR1_Mean  SINR1_Q5  RSRP1_Mean sq  nbrSq   gsq]
            
            # rs, sq로 데이터 정리하여 SISUL_CD, SIN1_STATE, RSRP1_STATE
            # input1 : REQUEST_DATE', 'SCENARIO_SEQ', 'REQUEST_SEQ', 'SISUL_CD, STC_STATE, NBR_STATE
            # SRC & NBR Cell tilt값에 대해 7단계로 구분하여 설정값 결정
            input1 = rq[['REQUEST_DATE', 'SCENARIO_SEQ', 'REQUEST_SEQ', 'SISUL_CD', 'SRC_STATE', 'NBR_STATE']]
            # input2 : 'SISUL_CD', 'SINR1_STATE', 'RSRP1_STATE
            # SRC Cell에 대한 N/W품질로 SINR, RSRP값을 7단계로 구분하여 설정값 결정
            input2 = a.cutStateQuality(rs)
            input = pd.merge(input1, input2, how = 'left', left_on = 'SISUL_CD', right_on= 'SISUL_CD')
            # ------------------------------------------------------------------------------

            steering_cell=pd.DataFrame([Cellcode], columns=['Cellcode'])
            gsq_avg=pd.merge(steering_cell, rs, how='left', left_on='Cellcode', right_on='SISUL_CD')['gsq'].mean()
            
            
            
            

            # 시점 t : action(tilt1 change val) 계산 ------------------------------------------------
            # 19개 셀에 대해서 Control Change값을 계산함
            # BO에서는 19개 셀의 Spec만을 계산하고
            # Request에서는 전체 셀의 Spec을 계산함
            # Action 반영 (임시) --> Target Cell(19개)의 Action 만 update함 

            # 수정필요함 : 현재는 68 cell에 대해 random값으로 설정함
            decAct = np.random.randint(-2, 2, rq.shape[0])        

            
            
            
            # 시점 t+1 : simulation실행하여 result파일생성후 -> 다음 iteration에서 reward와 state계산
            # SEQ+1차수의 Simulation실행을 위한 Request File 생성
            # SEQ차수의 Request File을 템플릿으로 사용함
            a.outputDatFile(action = decAct)  
            # -------------------------------------------------------------------------------
    
        # Request File이 생성되면 TEOS Simulation 실행 -> Result File이 나올때까지 실행, 오류발생하면 전체 실행 중단
        if Do_simul_NEXT(filetime, a)==1:
            a.seq=int(a.seq)+1
            print('File put과 get 명령이 완료되었습니다.')
        else:
            print('File put과 get에서 오류가 발생하여 중단합니다.')
            break
    # 객체 제거    
    del a

SyntaxError: invalid syntax (<ipython-input-827-55759943ab46>, line 86)

In [21]:
# def get_gsq_avg(filenm):
# 분석시마다 사용자가 입력해줘야 하는 값 ---------------------
# 시작 SEQ & 시간정보
# seq=44
# # 시뮬레이션 시작용 Request, Result 파일
# pre_reqfilenm='/home/tangosvc/code/data/20191030/SON_TO_TEOS_RL_REQUEST_20191030_[%s].dat' %str(seq)
# pre_resultfilenm='/home/tangosvc/code/data/20191030/TEOS_TO_SON_RL_RESULT_20191030_[%s].dat' %str(seq)
# # ------------------------------------------------------------    

seq=1030
timedata='20191112'
# 시뮬레이션 시작용 Request, Result 파일
pre_reqfilenm='/home/tangosvc/code/data/{}/SON_TO_TEOS_RL_REQUEST_{}_[{}].dat'.format(timedata, timedata, str(seq))
pre_resultfilenm='/home/tangosvc/code/data/{}/TEOS_TO_SON_RL_RESULT_{}_[{}].dat'.format(timedata, timedata, str(seq))

#값 초기화---------------------------------
# request, result file 관리 폴더(ML서버, TEOS서버)
ML_dirnm='/home/tangosvc/code/data'    

# simulation 실행일(파일생성일)
filetime=datetime.now().strftime('%Y%m%d') 
#----------------------------------------------  

# Request_filecreate 클래스를 생성함
a=Request_filecreate(ML_dirnm, filetime, seq)
a.pre_reqfilenm=pre_reqfilenm
a.pre_resultfilenm=pre_resultfilenm

rs, gsq_avg, gsq_each = a.getStateQuality(wa = 0.5, wb = 0.5)   

print(gsq_avg)
col1=pd.DataFrame(gsq_each)
col2=pd.DataFrame(cellcode)
print(col1, col2)

2.0879987760242886
         gsq
0   1.949101
1   2.827223
2   2.878098
3   2.136684
4   2.128772
5   1.616280
6   2.378363
7   2.236299
8   1.375752
9   2.063872
10  2.236299
11  0.616817
12  2.136684
13  1.846123
14  4.364633
15  0.616980             0
0   2019Y9667
1   2019Z2137
2   2019Z3737
3   20193BHZH
4   20193BHTY
5   20193AMR7
6   2019Y9678
7   2019Z3222
8   20193BIEF
9   20193BI6Z
10  2019Z2954
11  2019Z2586
12  2019Z0388
13  2019Z2764
14  2019Z2779
15  2019Y9862


/home/tangosvc/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)
